# Configuration

In [1]:
# Most straightforward method
import pyspark
import pandas as pd

def print_pandas(dataframe, n=5):
    return dataframe.limit(n).toPandas()

In [2]:
spark = pyspark.sql.SparkSession.builder \
        .master('local') \
        .appName('Spark Datatype') \
        .getOrCreate()

# Import data

In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

# If datatype is not specified, string will be default
squared = udf(lambda x: x**2)
# squared = udf(lambda x: x**2, FloatType())

In [4]:
df_pd = pd.DataFrame(
    data={'integers': [1, 2, 3], 
          'floats': [-1.0, 0.5, 2.7], 
          'integer_arrays': [[1, 2], [3, 4, 5], [6, 7, 8, 9]], 
          'strs': ['null', '', None]
         }
)

df = spark.createDataFrame(df_pd)
df.printSchema()

root
 |-- floats: double (nullable = true)
 |-- integer_arrays: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- integers: long (nullable = true)
 |-- strs: string (nullable = true)



In [5]:
print_pandas(df)

,floats,integer_arrays,integers,strs
0,-1.0,"[1, 2]",1,null
1,0.5,"[3, 4, 5]",2,
2,2.7,"[6, 7, 8, 9]",3,None


In [6]:
print_pandas(df.withColumn('floats_sq', squared('floats'))\
                 .withColumn('integers_sq', squared('integers')))

# df.withColumn('floats_sq', squared('floats'))\
#     .withColumn('integers_sq', squared('integers'))\
#     .printSchema()

,floats,integer_arrays,integers,strs,floats_sq,integers_sq
0,-1.0,"[1, 2]",1,null,1.0,1
1,0.5,"[3, 4, 5]",2,,0.25,4
2,2.7,"[6, 7, 8, 9]",3,None,7.290000000000001,9


In [7]:
print_pandas(df.select(squared('floats').alias('floats_sq'), 
                       squared('integers').alias('integers_sq')))

# df.select(squared('floats').alias('floats_sq'), 
#           squared('integers').alias('integers_sq')).printSchema()

,floats_sq,integers_sq
0,1.0,1
1,0.25,4
2,7.290000000000001,9


In [8]:
from pyspark.sql.types import ArrayType, FloatType

squared_list = udf(lambda x: [v**2 for v in x])

# # If you specific FloatType, but v is Integer, it will return None
# squared_list = udf(lambda x: [v**2 for v in x], ArrayType(FloatType()))
# squared_list = udf(lambda x: [float(v)**2 for v in x], ArrayType(FloatType()))

In [9]:
print_pandas(df.withColumn('integer_arrays_sq', squared_list('integer_arrays')))
# df.withColumn('integer_arrays_sq', squared_list('integer_arrays')).printSchema()

,floats,integer_arrays,integers,strs,integer_arrays_sq
0,-1.0,"[1, 2]",1,null,"[1, 4]"
1,0.5,"[3, 4, 5]",2,,"[9, 16, 25]"
2,2.7,"[6, 7, 8, 9]",3,None,"[36, 49, 64, 81]"


In [10]:
for col in df.columns:
    print(col)

floats
integer_arrays
integers
strs


In [13]:
for col in df.columns:
    print(type(df[col]))

<class 'pyspark.sql.column.Column'>
<class 'pyspark.sql.column.Column'>
<class 'pyspark.sql.column.Column'>
<class 'pyspark.sql.column.Column'>
